In [114]:
from pyspark.sql.functions import col, to_json, struct, when, lit
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
import os
import zipfile
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.functions import lit
import pyspark.sql.functions as F
from pyspark.sql.types import StringType,IntegerType
import cn2an
from random import sample
import math

In [115]:
def location(Choice):
    if Choice == '1':
        return '基隆市'
    elif Choice == '2':
        return '臺北市'
    elif Choice == '3':
        return '新北市'
    elif Choice == '4':
        return '桃園市'
    elif Choice == '5':
        return '新竹市'
    elif Choice == '6':
        return '新竹縣'
    elif Choice == '7':
        return '苗栗縣'
    elif Choice == '8':
        return '臺中市'
    elif Choice =='9':
        return '南投縣'
    elif Choice == '10':
        return '彰化縣'
    elif Choice == '11':
        return '雲林縣'
    elif Choice == '12':
        return '嘉義市'
    elif Choice == '13':
        return '嘉義縣'
    elif Choice == '14':
        return '臺南市'
    elif Choice == '15':
        return '高雄市'
    elif Choice == '16':
        return '屏東縣'
    elif Choice == '17':
        return '宜蘭縣'
    elif Choice == '18':
        return '花蓮縣'
    elif Choice == '19':
        return '臺東縣' 
    elif Choice == '20':
        return '澎湖縣'
    elif Choice == '21':
        return '金門縣'
    elif Choice == '22':
        return '連江縣' 


In [45]:
def File_location(Choice):
    if Choice == 'A':
        return '臺北市'
    elif Choice == 'F':
        return '新北市'
    elif Choice == 'H':
        return '桃園市'
    elif Choice == 'B':
        return '臺中市'
    elif Choice == 'E':
        return '高雄市'

In [10]:
def File_format(a):
    if a == '1':
        return 'XML 格式'
    
    elif a == '2':
        return 'TXT 格式'

    elif a == '3':
        return 'CSV 格式'

    elif a == '4':
        return 'XLS 格式'

In [15]:
def User_choice():
    Download = input('1.本期下載 2.非本期下載')

    if Download == '2':
        Download_type = input('1.全國 2.進階下載(可選擇縣市)') # 先確認使用者是否需選擇縣市
        if Download_type == '1':  #選擇只需要全國資料
            Year = input('請輸入資料年份')  #資料的年份
            Q    = input('請輸入第幾季')    #第幾季
            Format = input('1.XML 格式 2.TXT 格式 3.CSV 格式 4.XLS 格式') #選擇下載的格式
            return Download,Download_type,Year,Q,Format

        if Download_type == '2':  #選擇只需要全國資料

            Year = input('請輸入資料年份')  #資料的年份
            Q    = input('請輸入第幾季')    #第幾季
            Format = input('1.XML 格式 2.TXT 格式 3.CSV 格式 4.XLS 格式') #選擇下載的格式
            #【臺北市/新北市/桃園市/臺中市/高雄市】
            print('1.基隆市  2.臺北市  3.新北市  4.桃園市  5.新竹市 \n6.新竹縣  7.苗栗縣  8.臺中市  9.南投縣  10.彰化縣 \
                   \n11.雲林縣 12.嘉義市 13.嘉義縣 14.臺南市 15.高雄市 \n16.屏東縣 17.宜蘭縣 18.花蓮縣 19.臺東縣 20.澎湖縣\
                   \n21.金門縣 22.連江縣')
            local = []
            flag = 1
            while flag == 1:
                aa = input('您所要下載的縣市,若輸入完畢請輸入exit')
                
                if aa == 'exit':
                    flag = 0
                else:
                    local.append(aa)
            return Download,Download_type,Year,Q,Format,local

Choice = User_choice()

1.基隆市  2.臺北市  3.新北市  4.桃園市  5.新竹市 
6.新竹縣  7.苗栗縣  8.臺中市  9.南投縣  10.彰化縣                    
11.雲林縣 12.嘉義市 13.嘉義縣 14.臺南市 15.高雄市 
16.屏東縣 17.宜蘭縣 18.花蓮縣 19.臺東縣 20.澎湖縣                   
21.金門縣 22.連江縣


In [16]:
def Star_webdriver(Choice):
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    out_path = r"C:\Users\Darren.Kao\Desktop\code\02.Python code\cathay"
    prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': out_path} #下載檔案的路徑
    chrome_options.add_experimental_option('prefs', prefs)

    driver = webdriver.Chrome(executable_path=r"C:\Users\Darren.Kao\Desktop\code\02.Python code\cathay\chromedriver",chrome_options=chrome_options)
    driver.get('https://plvr.land.moi.gov.tw/DownloadOpenData') #透過chromedriver開啟網頁
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="ui-id-{}"]'.format(Choice[0])).click() # Choice[0] = 本期 OR 非本期
    time.sleep(3)
    if Choice[1] == '1' :
        select = Select(driver.find_element_by_name('season'))
        select.select_by_visible_text(u"{}年第{}季".format(Choice[2],Choice[3]))  #選擇年份 季
        time.sleep(3)
        Format  = File_format(Choice[4])
        select = Select(driver.find_element_by_name('fileFormat')) #XML TXT CSV XLS
        select.select_by_visible_text(Format)  #a[4]
        driver.find_element_by_xpath('//*[@id="downloadBtnId"]').click() 

    else : # 先選進階 #【臺北市/新北市/桃園市/臺中市/高雄市】
        driver.find_element_by_css_selector("input[type='radio'][value='{}']".format(Choice[1])).click()
        time.sleep(3)
        select = Select(driver.find_element_by_name('season'))
        select.select_by_visible_text(u"{}年第{}季".format(Choice[2],Choice[3]))  #選擇年份 季
        time.sleep(3)
        Format  = File_format(Choice[4])
        select = Select(driver.find_element_by_name('fileFormat')) #XML TXT CSV XLS
        select.select_by_visible_text(Format)  #a[4]

        for i in Choice[5]:
            localtion = location(i)
            aa = driver.find_elements_by_class_name("advDownloadClass")
            for i in aa :
                if i.text == localtion:
                    i.find_element_by_class_name("checkBoxGrp.landTypeA").click()
        driver.find_element_by_xpath('//*[@id="downloadBtnId"]').click()
    print('檔案下載中 請稍後....')
    time.sleep(20)
    
    # driver.close()
Star_webdriver(Choice)

檔案下載中 請稍後....


In [134]:
def zip_list(file_path): #解壓縮
    zf = zipfile.ZipFile(file_path, 'r')
    zf.extractall(r'C:\Users\Darren.Kao\Desktop\code\02.Python code\cathay\data')
path = r'C:\Users\Darren.Kao\Desktop\code\02.Python code\cathay\download.zip'
zip_list(path)

In [135]:
count = []
def DataRead():
    spark = SparkSession.builder.appName('HW Project').getOrCreate() #建立連接
    file_path = r'C:\Users\Darren.Kao\Desktop\code\02.Python code\cathay\data'
    allFileList = os.listdir(file_path)
    file = []
    for i in allFileList:  #過濾出要的檔案
        if 'lvr_land' in i :
            file.append(i)
    for i in range(len(file)) :
        if i == 0 :
            path = r'C:\Users\Darren.Kao\Desktop\code\02.Python code\cathay\data' + '/' + str(file[i])
            df_1 = spark.read.csv(path, #讀取檔案
                    sep=',',
                    header=True,
                    inferSchema=True
                    )
            local = File_location(file[i][0])
            count.append(local)
            df_1 = df_1.withColumn('city',lit('{}'.format(local)))
    
        else:
            path = r'C:\Users\Darren.Kao\Desktop\code\02.Python code\cathay\data' + '/'  + str(file[i])
            df_2 = spark.read.csv(path, #讀取檔案
                    sep=',',
                    header=True,
                    inferSchema=True
                    )
            local = File_location(file[i][0])
            count.append(local)
            df_2 = df_2.withColumn('city',lit('{}'.format(local)))
            df_1 = df_1.union(df_2) 

    return df_1


df = DataRead()

In [136]:
def Dataclean(df):
    """
    合併資料集,篩選出結果
    -【主要用途】為【住家用】
    -【建物型態】為【住宅大樓】
    -【總樓層數】需【大於等於十三層】
    解決方法 - > 先過濾出 住家用 住宅大樓 減少資料量 因樓層需另外處理

    """

    df = df.filter((df.建物型態.like('%住宅大樓%') ) &( df.主要用途=='住家用'))  #可透過 .like 找出符合條件的 


    """
    處理樓層 : 十三層(str) 轉為 13 (int) 轉換後才能進行過濾
    解決方法 - > 首先透過自訂之UDF將"層"這個字刪除  刪除後透過 cn2an 套件將中文轉阿拉伯數字 後進行過濾
    """

    UDF_1 = F.udf(lambda x: cn2an.cn2an(x.replace('層','')) if x != None else x,IntegerType()) 
    df = df.withColumn('新總樓層數',UDF_1(df.總樓層數))
    df = df.filter(df.新總樓層數 >= 13) 

    """
    處理時間格式 : 轉 Json 前須處理 交易日期  民國月日 -> 西元-月-日
    解決方法 - >  1080101 取字串 [前3位元(民國) + 1911 = 西元 ] + [第四位-最後一位] = 西元 + 年 + 月 = 20190101 再將格式轉乘 2019-01-01
    """
    UDF_2 = F.udf(lambda x: str(int(x[0:3])+1911)+x[3:],StringType())
    df = df.withColumn('交易年月日',UDF_2(df.交易年月日)).withColumn('交易年月日',F.to_date(F.unix_timestamp('交易年月日', 'yyyyMMdd').cast('timestamp')))
    df = df.sort(df.交易年月日.desc())

    """
    取出要轉成json格式之欄位
    """

    df = df.select([df.city,df.交易年月日,df.鄉鎮市區,df.建物型態])
    """
    轉成json格式
    """
    df = df.groupBy("city","交易年月日").agg(F.collect_list(struct("建物型態", "鄉鎮市區")).alias("events"))
    
    df = df.groupBy("city").agg(F.collect_list(struct("交易年月日", "events")).alias("time_slots"))
    df.show()
    return df 

a  = Dataclean(df)

In [139]:

file_count = int(input('使用者要分成幾個檔案?'))
b = math.ceil(len(count)/int(file_count))  #一個檔案放幾個
sum = 0
while(len(count) !=  0):
    for i in range(1,file_count+1): 
        while(sum < b):
            if len(count) == 0 :
                break
            else:
                with open("result-part{}.json".format(i), "a",encoding='utf-8') as f:
                    local = sample(count,1)[0]
                    print(local)
                    json = a.filter((F.col('city')=='{}'.format(local))).toJSON()
                    f.write(str(json.collect()))
                    f.write('\n')
                    count.remove(local)
                sum = sum +1
        sum = 0 